In [1]:
!pip install vosk
!wget https://alphacephei.com/vosk/models/vosk-model-ru-0.22.zip
!unzip vosk-model-ru-0.22.zip


--2025-07-13 05:57:00--  https://alphacephei.com/vosk/models/vosk-model-ru-0.22.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1559419132 (1.5G) [application/zip]
Saving to: ‘vosk-model-ru-0.22.zip.1’

vosk-model-ru-0.22. 100%[===================>]   1.45G  23.4MB/s    in 64s     

2025-07-13 05:58:05 (23.4 MB/s) - ‘vosk-model-ru-0.22.zip.1’ saved [1559419132/1559419132]

Archive:  vosk-model-ru-0.22.zip
replace vosk-model-ru-0.22/graph/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import os
import cv2
# import whisper
import torch
import json
from PIL import Image
from moviepy.editor import VideoFileClip
from transformers import BlipProcessor, BlipForConditionalGeneration
from datetime import datetime
from vosk import Model, KaldiRecognizer
import wave
import json as js
import os


# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs("extracted_frames", exist_ok=True)

# Load models
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
# whisper_model = whisUntitled7per.load_model("base")  # or 'small'/'medium'

# --------- Frame Extraction ----------
def extract_key_frames(video_path, num_frames=5):
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    indices = [int(i * total / num_frames) for i in range(num_frames)]
    paths = []
    for idx, f in enumerate(indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ret, frame = cap.read()
        if ret:
            name = os.path.splitext(os.path.basename(video_path))[0]
            path = f"extracted_frames/{name}_frame{idx}.jpg"
            cv2.imwrite(path, frame)
            paths.append(path)
    cap.release()
    return paths

# --------- Visual Captioning ----------
def generate_caption(img_path):
    image = Image.open(img_path).convert("RGB")
    inputs = blip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = blip_model.generate(**inputs)
    return blip_processor.decode(out[0], skip_special_tokens=True)

# # --------- Audio Transcription ----------
# def transcribe_audio(video_path):
#     temp_audio = "temp_audio.wav"
#     video = VideoFileClip(video_path)
#     video.audio.write_audiofile(temp_audio, verbose=False, logger=None)
#     result = whisper_model.transcribe(temp_audio)
#     os.remove(temp_audio)
#     return result["text"], result.get("segments", [])

# --------- File Metadata ----------
def get_file_info(path):
    stat = os.stat(path)
    return stat.st_size, datetime.utcfromtimestamp(stat.st_ctime).strftime('%Y-%m-%d %H:%M:%S UTC')

# --------- Combined Description ----------
def describe_video_multimodal(video_path, num_frames=5):
    # File metadata
    file_name = os.path.basename(video_path)
    file_path = os.path.abspath(video_path)
    file_size, created_at = get_file_info(video_path)
    clip = VideoFileClip(video_path)
    width, height = clip.size
    duration = clip.duration
    fps = clip.fps
    has_audio = clip.audio is not None
    fmt = file_name.split(".")[-1]

    # Visual description
    frame_paths = extract_key_frames(video_path, num_frames)
    visual_captions = [generate_caption(p) for p in frame_paths]

    # Audio transcription
    audio_text, audio_segments = transcribe_audio_vosk(video_path) if has_audio else ("", [])
    print("AUDIO", audio_text)
    # Merge content
    combined_desc = ". ".join(set(visual_captions)) + (". Spoken: " + audio_text if audio_text else "")

    return {
        "type": "video",
        "metadata": {
            "fileName": file_name,
            "filePath": file_path,
            "fileSize": file_size,
            "createdAt": created_at,
            "description": combined_desc[:150] + "...",
            "tags": list(set(word for cap in visual_captions for word in cap.lower().split() if word.isalpha()))[:5]
        },
        "duration": duration,
        "resolution": {"width": width, "height": height},
        "frameRate": fps,
        "hasAudio": has_audio,
        "videoFormat": fmt,
        "contentAnalysis": {
            "contentOverview": combined_desc,
            "actionIntroduction": visual_captions[0],
            "visualCaptions": visual_captions,
            "audioTranscript": audio_text,
            "audioSegments": audio_segments,
            "estimatedMood": "neutral"
        }
    }


# Load model once (globally)
vosk_model = Model("vosk-model-ru-0.22")

def transcribe_audio_vosk(video_path):
    temp_audio = "temp_audio.wav"
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(temp_audio, verbose=False, logger=None)

    wf = wave.open(temp_audio, "rb")
    rec = KaldiRecognizer(vosk_model, wf.getframerate())
    rec.SetWords(True)

    result_text = ""
    segments = []

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            part = js.loads(rec.Result())
            if "text" in part:
                result_text += part["text"] + " "
                segments.append(part)
    wf.close()
    os.remove(temp_audio)
    return result_text.strip(), segments


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



In [2]:
# video_paths = ["1.3.mp4", "2.5.mp4", "face.mp4"]
video_paths = ["tg_video_13_07.mp4"]

for path in video_paths:
    print(f"\nProcessing: {path}")
    try:
        result = describe_video_multimodal(path)
        with open(f"{os.path.splitext(os.path.basename(path))[0]}_multimodal_description.json", "w") as f:
            json.dump(result, f, indent=2)
        print("✅ Done:", result['metadata']['description'])
    except Exception as e:
        print("❌ Error:", e)



Processing: tg_video_13_07.mp4
AUDIO что да ты наташа что это почему же  сядь для сайт пошёл ты
✅ Done: a person sitting in a chair with a laptop. a woman with blonde hair and a black shirt. a woman with glasses is taking a self self self self self self ...


In [1]:
import os
import cv2
import torch
import json
import wave
import whisper
from PIL import Image
from moviepy.editor import VideoFileClip
from transformers import BlipProcessor, BlipForConditionalGeneration
from datetime import datetime

# ────────────────────────────────
# SETUP
# ────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs("extracted_frames", exist_ok=True)

# Load models
print("Loading models...")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

whisper_model = whisper.load_model("large-v3")
print("Models loaded.")

# ────────────────────────────────
# FRAME EXTRACTION
# ────────────────────────────────
def extract_key_frames(video_path, num_frames=5):
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    indices = [int(i * total / num_frames) for i in range(num_frames)]
    paths = []
    for idx, f in enumerate(indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ret, frame = cap.read()
        if ret:
            name = os.path.splitext(os.path.basename(video_path))[0]
            path = f"extracted_frames/{name}_frame{idx}.jpg"
            cv2.imwrite(path, frame)
            paths.append(path)
    cap.release()
    return paths

# ────────────────────────────────
# VISUAL CAPTIONING (BLIP)
# ────────────────────────────────
def generate_caption(img_path):
    image = Image.open(img_path).convert("RGB")
    inputs = blip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = blip_model.generate(**inputs)
    return blip_processor.decode(out[0], skip_special_tokens=True)

# ────────────────────────────────
# AUDIO TRANSCRIPTION (Whisper)
# ────────────────────────────────
def transcribe_audio_whisper(video_path):
    temp_audio = "temp_audio.wav"
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(temp_audio, verbose=False, logger=None)

    # Transcribe in Russian
    result = whisper_model.transcribe(temp_audio, language="ru")
    os.remove(temp_audio)

    return result["text"], result.get("segments", [])

# ────────────────────────────────
# FILE INFO UTILS
# ────────────────────────────────
def get_file_info(path):
    stat = os.stat(path)
    return stat.st_size, datetime.utcfromtimestamp(stat.st_ctime).strftime('%Y-%m-%d %H:%M:%S UTC')

# ────────────────────────────────
# FULL VIDEO DESCRIPTION
# ────────────────────────────────
def describe_video_multimodal(video_path, num_frames=5):
    file_name = os.path.basename(video_path)
    file_path = os.path.abspath(video_path)
    file_size, created_at = get_file_info(video_path)
    clip = VideoFileClip(video_path)
    width, height = clip.size
    duration = clip.duration
    fps = clip.fps
    has_audio = clip.audio is not None
    fmt = file_name.split(".")[-1]

    print(f"🖼 Extracting frames from {video_path}...")
    frame_paths = extract_key_frames(video_path, num_frames)
    visual_captions = [generate_caption(p) for p in frame_paths]

    print("🎙 Transcribing audio...")
    audio_text, audio_segments = transcribe_audio_whisper(video_path) if has_audio else ("", [])

    combined_desc = ". ".join(set(visual_captions))
    if audio_text:
        combined_desc += f". Spoken: {audio_text}"

    return {
        "type": "video",
        "metadata": {
            "fileName": file_name,
            "filePath": file_path,
            "fileSize": file_size,
            "createdAt": created_at,
            "description": combined_desc[:150] + "...",
            "tags": list(set(word for cap in visual_captions for word in cap.lower().split() if word.isalpha()))[:5]
        },
        "duration": duration,
        "resolution": {"width": width, "height": height},
        "frameRate": fps,
        "hasAudio": has_audio,
        "videoFormat": fmt,
        "contentAnalysis": {
            "contentOverview": combined_desc,
            "actionIntroduction": visual_captions[0] if visual_captions else "",
            "visualCaptions": visual_captions,
            "audioTranscript": audio_text,
            "audioSegments": audio_segments,
            "estimatedMood": "neutral"
        }
    }

# ────────────────────────────────
# PROCESS VIDEOS
# ────────────────────────────────
video_paths = ["tg_video_13_07.mp4"]  # Replace with your own

for path in video_paths:
    print(f"\n📦 Processing: {path}")
    try:
        result = describe_video_multimodal(path)
        with open(f"{os.path.splitext(os.path.basename(path))[0]}_multimodal_description.json", "w") as f:
            json.dump(result, f, indent=2, ensure_ascii=False)
        print("✅ Done:", result['metadata']['description'])
    except Exception as e:
        print("❌ Error:", e)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading models...


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Models loaded.

📦 Processing: tg_video_13_07.mp4
🖼 Extracting frames from tg_video_13_07.mp4...
🎙 Transcribing audio...


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



✅ Done: a woman with a black shirt and a white shirt. a woman with glasses is taking a self self self self self self self self self self self self self. a man...


In [4]:
!pip install -q git+https://github.com/openai/whisper.git
!sudo apt-get install -y ffmpeg


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [1]:
import whisper
import os
import cv2
import torch
import json
import wave
import whisper
from PIL import Image
from moviepy.editor import VideoFileClip
from transformers import BlipProcessor, BlipForConditionalGeneration
from datetime import datetime

# ────────────────────────────────
# SETUP
# ────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs("extracted_frames", exist_ok=True)


whisper_model = whisper.load_model("large-v3")
print("Models loaded.")


def transcribe_audio_whisper(video_path):
    temp_audio = "temp_audio.wav"
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(temp_audio, verbose=False, logger=None)

    # Transcribe in Russian
    result = whisper_model.transcribe(temp_audio, language="ru")
    os.remove(temp_audio)

    return result["text"], result.get("segments", [])


audio_path = "tg_video_13_07.mp4"
transcribe_audio_whisper(audio_path)

Models loaded.


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



(' скажи что-нибудь по-русски вон там код выполняет мое приложение используем себе протокол таким образом мне не приходится четко определять pipeline выполнение код код сам определяет в какой последовательности нужно что выполнять круто',
 [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 12.540000000000001,
   'text': ' скажи что-нибудь по-русски вон там код выполняет мое приложение используем',
   'tokens': [50365,
    21938,
    435,
    2143,
    12,
    23561,
    2801,
    12,
    2711,
    461,
    47399,
    740,
    1784,
    8223,
    981,
    1435,
    34771,
    5151,
    1094,
    1084,
    5805,
    47251,
    5627,
    15552,
    33096,
    50992],
   'temperature': 0.0,
   'avg_logprob': -0.187332245019766,
   'compression_ratio': 1.664804469273743,
   'no_speech_prob': 0.015318616293370724},
  {'id': 1,
   'seek': 0,
   'start': 12.540000000000001,
   'end': 23.16,
   'text': ' себе протокол таким образом мне не приходится четко определять pipeline выполнение код код